# Web Scrapping

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="myApp")
import haversine as hs

from statistics import mean
from haversine import Unit
import numpy as np
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

Getting address of the dental clinic, rating, reviews from google maps. I used selenium to scroll through all pages from the google maps automoatically by counting how many pages the search result had.

In [2]:
class WebDriver:

    def __init__(self):

        self.PATH = './chromedriver'
        self.options = Options()
        self.driver = webdriver.Chrome(self.PATH)
        self.location_lst = []

    def get_location_data(self):
        location_data = {}
        location_data["name"] = "NA"
        location_data["rating"] = "NA"
        location_data["reviews_count"] = "NA"
        location_data["location"] = "NA"
        location_data['exception'] = "NA"
        try:
            name = self.driver.find_element_by_xpath("//div[@class='SPZz6b']//span")
            avg_rating = self.driver.find_element_by_xpath("//div[@class='TLYLSe MaBy9']//span[@class='yi40Hd YrbPuc']")
            total_reviews = self.driver.find_element_by_xpath("//span[@class='NdWbqe Y0A0hc']//span[@class='RDApEe YrbPuc']")
            address = self.driver.find_element_by_xpath("//span[@class='LrzXr']")
            location_data["name"] = name.text
            location_data["rating"] = avg_rating.text
            location_data["reviews_count"] = total_reviews.text[1:-1]
            location_data["location"] = address.text

        except:
            name = self.driver.find_element_by_xpath("//div[@class='SPZz6b']//span")
            address = self.driver.find_element_by_xpath("//span[@class='LrzXr']")
            location_data["name"] = name.text
            location_data["location"] = address.text
            location_data['exception'] = "Error occured"

        return location_data
                                    
    def scrape(self, url): # Passed the URL as a variable

        self.driver.get(url) # Get is a method that will tell the driver to open at that particular URL
        time.sleep(7) # Waiting for the page to load.
        for i in range(11):
            results = self.driver.find_elements_by_xpath("//div[@class='dbg0pd']//span[@class='OSrXXb']")
            num_boxes = len(results)
            for j in range(3,num_boxes):
                try:
                    self.driver.find_elements_by_xpath("//div[@class='rllt__details']//span[@class='OSrXXb']")[j].click()
                    time.sleep(2)
                    loc_data = self.get_location_data()
                    self.location_lst.append(loc_data)
                    time.sleep(1)
                except:
                    pass

            self.driver.find_element_by_xpath("//span[@style='display:block;margin-left:53px']").click()
            time.sleep(1.5)
        self.driver.quit()

In [6]:
x = WebDriver()
url = "https://www.google.com/search?rlz=1C5CHFA_enUS967US967&tbs=lf:1,lf_ui:2&tbm=lcl&sxsrf=ALiCzsb63ThpCyYc7JrvXXQOQXCgZGZBcA:1668630378271&q=dental+clinics+in+waltham&rflfq=1&num=10&sa=X&ved=2ahUKEwjL3dTHxLP7AhXMGlkFHQ9zCuoQjGp6BAgkEAE&biw=1078&bih=978&dpr=0.9#rlfi=hd:;si:;mv:[[42.407184699999995,-71.1949413],[42.365521099999995,-71.2784801]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!1m4!1u2!2m2!2m1!1e1!2m1!1e2!2m1!1e3!3sIAE,lf:1,lf_ui:2"
x.scrape(url)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=108.0.5359.98)
Stacktrace:
0   chromedriver                        0x0000000104da6f38 chromedriver + 4910904
1   chromedriver                        0x0000000104d26a03 chromedriver + 4385283
2   chromedriver                        0x000000010496b747 chromedriver + 472903
3   chromedriver                        0x0000000104941ff5 chromedriver + 303093
4   chromedriver                        0x00000001049dab0f chromedriver + 928527
5   chromedriver                        0x00000001049f0763 chromedriver + 1017699
6   chromedriver                        0x00000001049d5ee3 chromedriver + 909027
7   chromedriver                        0x00000001049a030c chromedriver + 688908
8   chromedriver                        0x00000001049a188e chromedriver + 694414
9   chromedriver                        0x0000000104d741de chromedriver + 4702686
10  chromedriver   

Every census tract block has population data. Therefore, I linked censustract of each addresses to get the population for each clinics that is in the specific census tract and got census tract blocks and population of the blocks for each address.

In [18]:
tract_lst = []
pop_lst = []
def get_census_population(df, tract_lst, pop_lst):
    Path = './chromedriver'
    driver = webdriver.Chrome(Path)
    url = "https://geomap.ffiec.gov/FFIECGeocMap/GeocodeMap1.aspx"
    driver.get(url)
    time.sleep(5)

    for address in df["location"]:
        input_cell = driver.find_element_by_xpath("//div[@class='dijitReset dijitInputField dijitInputContainer']//input[@class='dijitReset dijitInputInner']")
        input_cell.send_keys(address)
        time.sleep(0.5)
        input_cell.send_keys(Keys.ENTER)
        time.sleep(3)
        tract_code = driver.find_element_by_xpath("//div[@id='TractCode']")
        tract_lst.append(tract_code.text)
        driver.find_element_by_xpath("//button[@id='btnCensusData1']").click()
        time.sleep(1)
        population = driver.find_element_by_xpath("//div[@id='cenPopulation']")
        pop_lst.append(population.text)
        driver.refresh()
        time.sleep(3)     
        
        

In [19]:
get_census_population(competitor_df, tract_lst, pop_lst)

/var/folders/zc/1dftl_rj0b55snyh5w747lv80000gn/T/ipykernel_87341/2604107497.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(Path)
/var/folders/zc/1dftl_rj0b55snyh5w747lv80000gn/T/ipykernel_87341/2604107497.py:11: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  input_cell = driver.find_element_by_xpath("//div[@class='dijitReset dijitInputField dijitInputContainer']//input[@class='dijitReset dijitInputInner']")
/var/folders/zc/1dftl_rj0b55snyh5w747lv80000gn/T/ipykernel_87341/2604107497.py:16: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  tract_code = driver.find_element_by_xpath("//div[@id='TractCode']")
/var/folders/zc/1dftl_rj0b55snyh5w747lv80000gn/T/ipykernel_87341/2604107497.py:18: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH

Inserting population and census tract data into the dataframe and saving it to csv file for later usage.

In [29]:
competitor_df["Population"] = pop_lst
competitor_df["Census_Tract"] = tract_lst
competitor_df.to_csv("competitor_info4.csv")

In [211]:
competitor_df = pd.DataFrame(x.location_lst)

In [217]:
competitor_df.to_csv("competitor_info.csv")

Get coordinates latitude and longitude for the data to calculate average distance

In [280]:
coor_lst = []
def get_coor(df,coor_lst):
    path = './chromedriver'
    driver = webdriver.Chrome(path)
    url = "https://gps-coordinates.org/"
    driver.get(url)
    time.sleep(4)
    for address in df["location"]:
        input_element = driver.find_element_by_xpath("//input[@id='address']")
        input_element.clear()
        time.sleep(0.5)
        input_element.send_keys(address)
        time.sleep(0.5)
        search_element = driver.find_element_by_xpath("//button[@id='btnGetGpsCoordinates']")
        search_element.click()
        time.sleep(1)
        lat_input = driver.find_element_by_xpath("//input[@id='latitude']")
        longt_input = driver.find_element_by_xpath("//input[@id='longitude']")
        lat = lat_input.get_attribute("value")
        longt = longt_input.get_attribute("value")
        coor_lst.append((lat,longt))
        time.sleep(1)

In [281]:
get_coor(competitor_df, coor_lst)

/var/folders/zc/1dftl_rj0b55snyh5w747lv80000gn/T/ipykernel_36888/4168279071.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)
/var/folders/zc/1dftl_rj0b55snyh5w747lv80000gn/T/ipykernel_36888/4168279071.py:9: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  input_element = driver.find_element_by_xpath("//input[@id='address']")
/var/folders/zc/1dftl_rj0b55snyh5w747lv80000gn/T/ipykernel_36888/4168279071.py:14: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  search_element = driver.find_element_by_xpath("//button[@id='btnGetGpsCoordinates']")
/var/folders/zc/1dftl_rj0b55snyh5w747lv80000gn/T/ipykernel_36888/4168279071.py:17: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  lat_input = driver.find_element_by_xpat

In [293]:
lat_long_lst = []
for i in coor_lst:
    lat_long = [float(j) for j in i]
    lat_long_lst.append(lat_long)

62

Insert latitude and longitude data into the dataframe

In [314]:
com_df["Latitude"] = np.NAN
com_df["Longitude"] = np.NAN

In [315]:
com_df = pd.read_csv("competitor_info.csv")

In [316]:
for index, item in enumerate(lat_long_lst):
    com_df["Latitude"].iloc[index] = item[0]
    com_df["Longitude"].iloc[index] = item[1]

KeyError: 'Latitude'

In [327]:
dist = []
for i in comp_df.index :
    lt_lg = (comp_df['Latitude'][i], comp_df['Longitude'][i])
    dist.append(lt_lg)

[(42.3764128, -71.2462826),
 (42.4549959, -71.0655774),
 (42.3693348, -71.2187824),
 (42.3757599, -71.2421873),
 (42.3764128, -71.2462826),
 (42.3677572, -71.2377956),
 (42.4045948, -71.2172707),
 (42.3748086, -71.2497742),
 (42.4049617, -71.2347331),
 (42.3697718, -71.2369958),
 (42.3869904, -71.1993747),
 (42.3747041, -71.234192),
 (42.394863, -71.261388),
 (42.3708125, -71.2367487),
 (42.3766362, -71.2737223),
 (42.3696466, -71.2481156),
 (42.3747041, -71.234192),
 (42.372369, -71.2369462),
 (42.3760918, -71.250963),
 (42.3916135, -71.2565512),
 (42.3916135, -71.2565512),
 (42.3763172, -71.2378067),
 (42.3695497, -71.2489404),
 (42.3953336, -71.2636731),
 (42.3768165, -71.231317),
 (42.3761225, -71.2135514),
 (42.4059669, -71.2332125),
 (42.3696466, -71.2481156),
 (42.4045948, -71.2172707),
 (42.3748086, -71.2497742),
 (42.3704565, -71.2383092),
 (42.4119322, -71.2329416),
 (42.3748086, -71.2497742),
 (42.3764128, -71.2462826),
 (42.3696466, -71.2481156),
 (42.3760286, -71.2529152),

Get average distance to other competitors location from the selected location using haversine library

In [329]:
# Get average distance to other competitors location from the selected location
# use haversine library
distance_lst = []
for loc2 in dist:
    distance_lst.append(hs.haversine(open_location_a,loc2, unit=Unit.MILES))

In [338]:
avg_dist = []
def get_avg(dist):
    distance_lst = []
    i = 0
    while i < len(dist):
        distan = [hs.haversine(dist[i],dist[j], unit=Unit.MILES) for j in range(1, len(dist))]
        distance_lst.append(mean(distan))
        i += 1
    return distance_lst

In [331]:
distance_lst = []
for i in range(len(dist)):
    for j in range(1, len(dist)):
        distance_lst.append(hs.haversine(dist[i],dist[j], unit=Unit.MILES))

In [342]:
avg_dist = get_avg(dist)
comp_df["Average_Distance"] = avg_dist

Preprocessing unnecessary words in the dataset that doesn't work with geopandas library

In [89]:
import re
remove_lst = ["UNIT","Unit","3rd", "#", "Suite", "DOCTORS OFFICE BUILDING", "STE", "Ste", "Dental Department", "Pet Supplies Plus"]
cleaned_lst = []
for i in competitor_df["location"]:
    reduced = [s for s in i.split(',')]
    # print(reduced)
    for index,element in enumerate(reduced):
        # print(element)
        if "DOCTORS OFFICE BUILDING" == element or "Dental Department" == element or "Pet Supplies Plus" == element:
            removed_element = reduced.pop(0)
    for j in remove_lst:
        if j in reduced[0]:
            reduced[0] = reduced[0].split(j)[0].rstrip(" ")
    cleaned_lst.append(",".join(reduced))

competitor_df["location"] = cleaned_lst
    

In [94]:
competitor_df = competitor_df.drop(176)

In [96]:
competitor_df.drop(171, inplace = True)

In [98]:
def eval_results(x):
    try:
        return (x.latitude, x.longitude)
    except:
        return (None, None)
competitor_df[['lat_long']] = competitor_df['location'].apply(geolocator.geocode, timeout=1000000).apply(lambda x: eval_results(x))